In [1]:
%reset -f

In [2]:
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn 
import torch.optim as optim
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import optuna
# from optuna.trial import TrialState
import os
import joblib
import torch.nn.functional as F

In [9]:
start_list = [0,1,2,3,4,5,0]
number_of_corner_list = [1,2,3,4,5,6,6]
feature_list = [0, 1]
truncate_number = 60
for i in range(1):
    feature = feature_list[i]
    for i in range(1):
        start = start_list[i]
        number_of_corner = number_of_corner_list[i]
        df_begin = []
        num_row = []
        num_row_corner = []
        percent = []
    #     start = 0s
    #     number_of_corner = 1
        f_1 = 'beginner_expert_processedData/beginner/beginner_'
        f_3 = '.csv'
        num_begin = 19
#         curveList = [[103.9, 209.3], [316.6, 399.6], [425.3, 517.9], [590.5, 756.9], [1212.3, 1437.1]]
        curveList = [[103.9, 209.3], [316.6, 399.6], [425.3, 517.9], [590.5, 756.9], [1048.7, 1110.5], [1212.3, 1437.1]]
#         curveList = [[103.9, 156.6], [156.6, 209.3], [316.6, 358.1], [358.1, 399.6], 
#                      [425.3, 471.6], [471.6, 517.9],p[590.5, 673.7], [673.7, 756.9],
#                      [1048.7, 1079.6], [1079.6, 1110.5], [1212.3, 1324.7], [1324.7, 1437.1]]

        df_concat = pd.DataFrame()
        for curve_num in range(start,number_of_corner):
            for idx in range(1, num_begin+1):
                tmp_file = f_1+str(idx)+'_new2'+f_3
                df = pd.read_csv(tmp_file)
                df = df.dropna()
                tmp = df.astype(float)
                tmp['level'] =0
                tmp['curve_number'] = curve_num
                tmpcorner = tmp[(tmp['Distance'] >= curveList[curve_num][0]) & (tmp['Distance'] <= curveList[curve_num][1])]
                num_row.append(np.size(tmpcorner,0)) 
                num_row_corner.append(np.size(tmpcorner,0)) 
                df_begin.append(tmpcorner)
                df_concat = pd.concat([df_concat,df_begin[idx-1]])      
            df_concat.to_csv('cornerData2/corner_'+str(curve_num+1)+'_begin'+'.csv')
            df_concat = pd.DataFrame()
            df_begin = []
        df_exp = []
        f_1 = 'beginner_expert_processedData/expert/expert_'
        f_3 = '.csv'
        num_exp = 19
        
        
        df_concat = pd.DataFrame()
        for curve_num in range(start,number_of_corner):
            for idx in range(1, num_exp+1):
                tmp_file = f_1+str(idx)+'_new2'+f_3
                df = pd.read_csv(tmp_file)
                df = df.dropna()
                tmp = df.astype(float)
                tmp['level'] =1
                tmp['curve_number'] = curve_num
                tmpcorner = tmp[(tmp['Distance'] >= curveList[curve_num][0]) & (tmp['Distance'] <= curveList[curve_num][1])]
                num_row.append(np.size(tmpcorner,0))
                num_row_corner.append(np.size(tmpcorner,0)) 
                df_exp.append(tmpcorner)
                df_concat = pd.concat([df_concat,df_exp[idx-1]])
            df_concat.to_csv('cornerData2/corner_'+str(curve_num+1)+'_expert'+'.csv')
            df_concat = pd.DataFrame()
            df_exp = []
            num_row_corner = np.array(num_row_corner)
            per = 60  ##truncate to 60
            percent.append(per)
            num_row_corner = []
        sequence_length = min(num_row)
        if sequence_length < truncate_number:
            sequnce_length = truncate_number + 5
        mean_row = round(np.mean(num_row))
        mean_row = min(num_row)

        if feature == 0:
            left_column = [
                'curve_number',
                'GPS Latitude','GPS Longitude',
                'CG Distance',
                'Damper Velocity (Calc) FL','Damper Velocity (Calc) FR','Damper Velocity (Calc) RL',
            'Damper Velocity (Calc) RR','Corr Speed','Brake Pos',
            'CG Accel Lateral','CG Accel Longitudinal','CG Accel Vertical','CG Height','Camber FL','Camber FR','Camber RL','Camber RR','Car Coord X',
            'Car Coord Y','Car Coord Z','Car Pos Norm','Chassis Pitch Angle','Chassis Pitch Rate','Chassis Roll Angle','Chassis Roll Rate',
            'Chassis Velocity X','Chassis Velocity Y','Chassis Velocity Z','Chassis Yaw Rate','Drive Train Speed','Engine RPM','Ground Speed',
            'Ride Height FL','Ride Height FR','Ride Height RL','Ride Height RR','Road Temp','Self Align Torque FL','Self Align Torque FR',
            'Self Align Torque RL','Self Align Torque RR','Steering Angle','Suspension Travel FL','Suspension Travel FR',
            'Suspension Travel RL','Suspension Travel RR','Tire Load FL','Tire Load FR','Tire Load RL','Tire Load RR','Tire Loaded Radius FL',
            'Tire Loaded Radius FR','Tire Loaded Radius RL','Tire Loaded Radius RR','Tire Pressure FL','Tire Pressure FR','Tire Pressure RL','Tire Pressure RR',
            'Tire Rubber Grip FL','Tire Rubber Grip FR','Tire Rubber Grip RL','Tire Rubber Grip RR','Tire Slip Angle FL','Tire Slip Angle FR',
            'Tire Slip Angle RL','Tire Slip Angle RR','Tire Slip Ratio FL','Tire Slip Ratio FR','Tire Slip Ratio RL','Tire Slip Ratio RR',
            'Tire Temp Core FL','Tire Temp Core FR','Tire Temp Core RL','Tire Temp Core RR','Tire Temp Inner FL','Tire Temp Inner FR',
            'Tire Temp Inner RL','Tire Temp Inner RR','Tire Temp Middle FL','Tire Temp Middle FR','Tire Temp Middle RL',
            'Tire Temp Middle RR','Tire Temp Outer FL','Tire Temp Outer FR','Tire Temp Outer RL','Tire Temp Outer RR','Toe In FL',
            'Toe In FR','Toe In RL','Toe In RR','Wheel Angular Speed FL','Wheel Angular Speed FR','Wheel Angular Speed RL','Wheel Angular Speed RR',
            'Lateral Velocity','Longitudinal Velocity','Lateral Acceleration','Longitudinal Acceleration','level']

        else:
            left_column = ['curve_number','Brake Pos', 'Ground Speed', 'Steering Angle', 'Throttle Pos', 'Chassis Yaw Rate', 'Chassis Velocity X',
                               'Chassis Velocity Y','Chassis Velocity Z','Lateral Velocity','Longitudinal Velocity','Lateral Acceleration','Longitudinal Acceleration',
                               'CG Distance',
                           'level']

        #Hyper-parameters
        num_epochs = 50
        batches = 1
        learning_rate = 0.001
        input_size = len(left_column)-1 # left column except 'level'
        output_size = 2 # Expert and Beginner
        hidden_size = 100 # ?
        num_layers = 2
        num_begin_train = round(num_begin*0.60)*(number_of_corner-start)
        num_exp_train = round(num_exp*0.60)*(number_of_corner-start)
        num_begin_test = num_begin*(number_of_corner-start) - num_begin_train
        num_exp_test = num_exp*(number_of_corner-start) - num_exp_train
        aug = 1

        
        class GRU(nn.Module):
            def __init__(self, input_size, hidden_size, num_layer, output_size):
                super(GRU, self).__init__()
                self.num_layers = num_layers
                self.hidden_size = hidden_size
                self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
                self.fc = nn.Linear(hidden_size, output_size)

            def forward(self, x):
                h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
                c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
                out, _ = self.gru(x, h0)
                out = out[:, -1, :]
                out = self.fc(out)
                return out
        

        ## Data Processing
        array_x = []
        array_y = []
        input_x = []
        input_y = []
        n_row = []

        df_tmp_begin = pd.DataFrame() 
        df_tmp_exp = pd.DataFrame() 
        for curve_num in range(start,number_of_corner):
            df_tmp_begin = pd.concat([df_tmp_begin,pd.read_csv('cornerData2/corner_'+str(curve_num+1)+'_begin.csv')])
            df_tmp_exp   = pd.concat([df_tmp_exp,pd.read_csv('cornerData2/corner_'+str(curve_num+1)+'_expert.csv')])    
        df_curve1 = pd.concat([df_tmp_begin, df_tmp_exp], ignore_index=True) 
        df_curve1 = df_curve1.loc[:,left_column]
        df_curve1_saved = df_curve1.loc[:,left_column] # data backup
        df_curve1.to_csv('cornerData2/corner_'+'_dfcurve1'+'.csv')
        datum = df_curve1_saved
        yyy = datum.pop('level')
        left = left_column.remove('level')
        for i in range(0,num_begin*(number_of_corner-start) + num_exp*(number_of_corner-start)):
            y = yyy.loc[0:num_row[i]-1]
            x_original = datum.loc[0:num_row[i]-1]
            scaler = MinMaxScaler()
            x_normal = scaler.fit_transform(x_original)
            x_normal = scaler.transform(x_original)
            x_normal = np.pad(x_normal,[(0,60),(0,0)],'edge') #post padding ## edge padding
            x = pd.DataFrame(x_normal,columns=left)
            p = i//(num_begin*2)
            x = x.truncate(after=percent[p]-1)
            datum.drop(range(0,num_row[i]),inplace=True)
            datum.reset_index(drop=True, inplace=True)
            yyy.drop(range(0,num_row[i]),inplace=True)
            yyy.reset_index(drop=True, inplace=True)
            array_x.append(x)
            array_y.append(y)



        ## Randomize sequence 
        # sequence = np.arange((num_begin*(number_of_corner-start) + num_exp*(number_of_corner-start))*aug/2)
        seq_train_begin = np.arange(num_begin_train)
        seq_test_begin = np.arange(num_begin_test) + num_begin_train
        seq_train_exp = seq_train_begin + num_begin*(number_of_corner-start)
        seq_test_exp = seq_test_begin + num_begin*(number_of_corner-start)
        print(seq_train_begin,seq_train_exp)
        np.random.seed(13)
        np.random.shuffle(seq_train_begin)
        np.random.seed(13)
        np.random.shuffle(seq_test_begin)
        np.random.seed(13)
        np.random.shuffle(seq_train_exp)
        np.random.seed(13)
        np.random.shuffle(seq_test_exp)


        sequence = np.concatenate((seq_train_begin, seq_train_exp, seq_test_begin, seq_test_exp), axis=None)
        sequence = sequence.astype('int')
        print(sequence)


        for i in sequence:
            input_x.append(array_x[i])
            input_y.append(array_y[i])
            p = i//(num_begin*2)
            n_row = n_row + [percent[p]]
        
    
            
        def train_mnist(trial):
            
            d2 = {'test sample': [], 'loss': [], 'accuracy': [], 'epoch' : [], 'batch':[],'learning rate' :[], 'hidden size':[],'hidden_layer':[]}
            df2 = pd.DataFrame(data=d2)
            loss_list = []
            iteration_list = []
            accuracy_list = []
            test_list=[]
            accuracy2_list=[]
            count = 0
            cfg = { 'device' : "cuda" if torch.cuda.is_available() else "cpu",
              'train_batch_size' : 64,
              'test_batch_size' : 1000,
              'n_epochs' : 1,
              'seed' : 0,
              'log_interval' : 100,
              'save_model' : False,
              'lr'       : 0.0008674236780492814,          
              'hidden_size' : 126,
#               'lr'       : trial.suggest_loguniform('lr', 5e-4, 5e-3),          
#               'hidden_size' : trial.suggest_int('hidden_size', 60, 200),
              'optimizer': optim.Adam,
              'activation': F.relu}
        
            torch.manual_seed(cfg['seed'])
        
            gru = GRU(input_size, cfg['hidden_size'], num_layers, output_size)
            criterion = nn.CrossEntropyLoss()
#         optimizer = torch.optim.Adam(gru.parameters(), lr=learning_rate)       
            optimizer = cfg['optimizer'](gru.parameters(), lr=cfg['lr'])
    
    
            for epoch in range(num_epochs):
                for i in range(0,(num_begin_train + num_exp_train)*aug):
                    # array type (numpy) 앞
                    X = np.array(input_x[i])
                    X = X.reshape(-1,n_row[i],input_size)
                    Y = np.array(input_y[i])

                    # tensor type (pytorch)
                    X = torch.from_numpy(X)
                    X = X.float()
                    Y = torch.tensor([Y[0]])
                    Y = Y.type(torch.LongTensor)
                    optimizer.zero_grad()
                    output = gru(X)
                    loss = criterion(output, Y)

                    # Backward and optimize
            #         optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                count += 1
                loss_list.append(loss.data)
                iteration_list.append(count)
                print (f'Epoch: [{epoch}/{num_epochs}]' f'Loss: {loss.item():.4f}')
            ## Test
                with torch.no_grad():
                    n_correct = 0
                    n_correct2 = 0
                    n_samples = 0
                    n_samples2 = 0

                    for i in range((num_begin_train + num_exp_train)*aug, (num_begin*(number_of_corner-start) + num_exp*(number_of_corner-start))*aug-4):

                        # array type (numpy)
                        X = np.array(input_x[i])
                        X = X.reshape(-1,n_row[i],input_size)
                        Y = np.array(input_y[i])

                        # tensor type (pytorch)
                        X = torch.from_numpy(X)
                        X = X.float()
                        Y = torch.tensor([Y[0]])
                        Y = Y.type(torch.LongTensor)
                        output = gru(X)
                        _, predicted = torch.max(output.data, 1)
                        n_samples += Y.size(0)
                        n_correct += (predicted == Y).sum().item()

    
                    for i in range((num_begin*(number_of_corner-start) + num_exp*(number_of_corner-start))*aug-4,(num_begin*(number_of_corner-start) + num_exp*(number_of_corner-start))*aug):

                        # array type (numpy)
                        X = np.array(input_x[i])
                        X = X.reshape(-1,n_row[i],input_size)
                        Y = np.array(input_y[i])

                        # tensor type (pytorch)
                        X = torch.from_numpy(X)
                        X = X.float()
                        Y = torch.tensor([Y[0]])
                        Y = Y.type(torch.LongTensor)
                        output = gru(X)
                        _, predicted2 = torch.max(output.data, 1)
                        n_samples2 += Y.size(0)
                        n_correct2 += (predicted2 == Y).sum().item()
    
    
                    acc = 100.0 * n_correct / n_samples
                    acc2 = 100.0 * n_correct2 / n_samples2
                    accuracy_list.append(acc)
                    accuracy2_list.append(acc2)
                    print(f'Accuracy of the network on the {(num_begin_test + num_exp_test)*aug} test images: {acc} %')          
            plt.plot(iteration_list,loss_list)
            plt.xlabel("Number of iteration")
            plt.ylabel("Loss")
            plt.title("GRU: Loss vs Number of iteration")

            if feature == 0:
                plt.savefig(f'cornerData2/loss_VS_epoch_corner{number_of_corner}_{number_of_corner-start}_allFeat1{curve_num}.png')
                plt.clf()
            else:
                plt.savefig(f'cornerData2/loss_VS_epoch_corner{number_of_corner}_{number_of_corner-start}_selectedFeat1{curve_num}.png')
                plt.clf()

            plt.plot(iteration_list,accuracy_list)
            plt.xlabel("Number of iteration")
            plt.ylabel(" Validation Accuracy")
            plt.title("GRU: Validaiton Accuracy vs Number of iteration")

            if feature == 0:
                plt.savefig(f'cornerData2/loss_VS_epoch_corner{number_of_corner}_{number_of_corner-start}_allFeat1_acc{curve_num}.png')
                plt.clf()
            else:
                plt.savefig(f'cornerData2/loss_VS_epoch_corner{number_of_corner}_{number_of_corner-start}_selectedFeat1_acc{curve_num}.png')
                plt.clf()    

            plt.plot(iteration_list,accuracy2_list)
            plt.xlabel("Number of iteration")
            plt.ylabel(" Test Accuracy")
            plt.title("GRU: Test Accuracy vs Number of iteration")

            if feature == 0:
                plt.savefig(f'cornerData2/loss_VS_epoch_corner{number_of_corner}_{number_of_corner-start}_allFeat1_acc{curve_num}.png')
                plt.clf()
            else:
                plt.savefig(f'cornerData2/loss_VS_epoch_corner{number_of_corner}_{number_of_corner-start}_selectedFeat1_acc{curve_num}.png')
                plt.clf()  

            print(f'Accuracy of the network on the {num_begin_test + num_exp_test} test images: {acc} %')
            print(f'Loss: {loss.item():.4f}')
            return acc

        study = optuna.create_study(direction='maximize')
        study.optimize(train_mnist, n_trials=1)
        joblib.dump(study, f'cornerData2/mnist_optuna{curve_num}.pkl')
    
        study = joblib.load(f'cornerData2/mnist_optuna{curve_num}.pkl')
        df3 = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
        print(df3)






    

[I 2021-06-19 22:08:27,344] A new study created in memory with name: no-name-8f0d9c61-539d-4bab-9e0e-6f3c6245ef22


[ 0  1  2  3  4  5  6  7  8  9 10] [19 20 21 22 23 24 25 26 27 28 29]
[ 3  5  7  8  1  4 10  6  9  0  2 22 24 26 27 20 23 29 25 28 19 21 12 15
 14 16 17 18 11 13 31 34 33 35 36 37 30 32]
Epoch: [0/100]Loss: 0.8999
Accuracy of the network on the 16 test images: 58.333333333333336 %
Epoch: [1/100]Loss: 0.6998
Accuracy of the network on the 16 test images: 100.0 %
Epoch: [2/100]Loss: 0.6106
Accuracy of the network on the 16 test images: 83.33333333333333 %
Epoch: [3/100]Loss: 0.5772
Accuracy of the network on the 16 test images: 91.66666666666667 %
Epoch: [4/100]Loss: 0.5216
Accuracy of the network on the 16 test images: 91.66666666666667 %
Epoch: [5/100]Loss: 0.4270
Accuracy of the network on the 16 test images: 91.66666666666667 %
Epoch: [6/100]Loss: 0.2945
Accuracy of the network on the 16 test images: 91.66666666666667 %
Epoch: [7/100]Loss: 0.1420
Accuracy of the network on the 16 test images: 91.66666666666667 %
Epoch: [8/100]Loss: 0.0528
Accuracy of the network on the 16 test images

Epoch: [95/100]Loss: 0.0000
Accuracy of the network on the 16 test images: 100.0 %
Epoch: [96/100]Loss: 0.0000
Accuracy of the network on the 16 test images: 100.0 %
Epoch: [97/100]Loss: 0.0000
Accuracy of the network on the 16 test images: 100.0 %
Epoch: [98/100]Loss: 0.0000
Accuracy of the network on the 16 test images: 100.0 %
Epoch: [99/100]Loss: 0.0000
Accuracy of the network on the 16 test images: 100.0 %


[I 2021-06-19 22:09:34,930] Trial 0 finished with value: 100.0 and parameters: {}. Best is trial 0 with value: 100.0.


Accuracy of the network on the 16 test images: 100.0 %
Loss: 0.0000
   number  value               duration
0       0  100.0 0 days 00:01:07.585592


<Figure size 432x288 with 0 Axes>

In [4]:
# # df2.to_csv('cornerData/result_gru.csv')
# # d2 = {'test sample': [], 'loss': [], 'accuracy': [], 'epoch' : [], 'batch':[],'learning rate' :[], 'hidden size':[],'hidden_layer':[]}
# # df2 = pd.DataFrame(data=d2)
# d = {'test sample': [num_begin_test + num_exp_test], 'loss': [loss.item()], 'accuracy': [acc], 'epoch' : [num_epochs], 'batch':[batches],'learning rate' :[learning_rate], 'hidden size':[hidden_size],'hidden_layer':[num_layers]}
# df = pd.DataFrame(data=d)
# df2 = df2.append(df+4)
# print(df2)

In [5]:
# x_normal = np.pad(x_normal,[(0,sequence_length-num_row[i]),(0,0)],'edge')
# print(x_normal)
# percent.append(60)
# print(percent)
print(sequence_length-num_row[1])
print(x_normal)
print(num_row)

-21
[[0.         1.         1.         ... 0.01574803 0.91111111 0.65517241]
 [0.         0.99451124 0.98900871 ... 0.09448819 0.86666667 0.51724138]
 [0.         0.98849974 0.97787916 ... 0.03149606 0.77777778 0.34482759]
 ...
 [0.         0.         0.         ... 0.92125984 0.33333333 0.34482759]
 [0.         0.         0.         ... 0.92125984 0.33333333 0.34482759]
 [0.         0.         0.         ... 0.92125984 0.33333333 0.34482759]]
[111, 98, 105, 88, 115, 105, 112, 95, 94, 108, 127, 140, 135, 99, 97, 86, 90, 111, 97, 87, 86, 86, 84, 81, 81, 111, 88, 80, 78, 77, 77, 84, 80, 77, 79, 79, 80, 84]


In [6]:
df3 = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)

In [7]:
for curve_num in range(6):
    study = joblib.load(f'cornerData2/mnist_optuna{curve_num}.pkl')
    df3 = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
    print(df3)
    df3=0
    

FileNotFoundError: [Errno 2] No such file or directory: 'cornerData2/mnist_optuna0.pkl'

In [ ]:
print((num_begin_train + num_exp_train)*aug, (num_begin*(number_of_corner-start) + num_exp*(number_of_corner-start))*aug)

In [ ]:
gru = GRU(input_size, cfg['hidden_size'], num_layers, output_size)